Shalagin Danil

In [1]:
import simpy
import queue
import numpy as np

# Task 4

On average, 3 cars arrive at the warehouse per hour. Unloading is carried out by 3 teams of loaders. The average unloading time of the machine is 1 hour. There can be no more than 4 cars in the queue waiting for unloading. Determine the performance of the queuing system.

In [2]:
class Warehouse:
    def __init__(self,env):
        self.num_servers = 3
        self.env = env
        self.cashier = simpy.Resource(env,self.num_servers)
        self.mu = 1
        
        self.queue_lenght = 0
        self.req_den = 0
        self.req_served_im = 0
        self.cur_req_num = 0
        self.req_num = 0
        
        self.start_b_time = 0
    def is_free(self,time):
        if self.cashier.count==0:
            busy_time  = time - self.start_b_time
            self.start_b_time = time
            return busy_time if not (busy_time==0) else None
        else:
            return None
        
    def unload(self,client):
        time = round(np.random.exponential(self.mu),2)
        #print(f"mu {time}")
        yield self.env.timeout(time)
        
    def add_to_queue(self):
        if self.queue_lenght<4:
            self.queue_lenght+=1
            #print("add")
            #print(self.queue_lenght)
            return True
        else:
            return False
    def remove_from_queue(self):
        if self.queue_lenght>0:
            self.queue_lenght-=1

In [3]:

def go_to_warehouse(env,client,warehouse,s):
    time_arrive = env.now
    collect_metrics(warehouse.queue_lenght+3-warehouse.cashier.count+1,"avrg_req_num",s)# collect queue
    #print(f"go {warehouse.queue_lenght}")
    if warehouse.cashier.count==warehouse.num_servers:
        if warehouse.add_to_queue():
            collect_metrics(warehouse.queue_lenght,"q_length",s)# collect queue
            #print(f"Car {client} in queue at time {env.now}")
            
            with warehouse.cashier.request() as request:
                #print(f"Car {client} request resourse at time {env.now}")
                time_leave_q = env.now
                warehouse.cur_req_num+=1
                warehouse.req_num+=1
                collect_metrics(warehouse.cur_req_num/client,"cur_n",s)
                collect_metrics(warehouse.req_num/client,"util",s)
                
                yield request
                warehouse.remove_from_queue()

                yield env.process(warehouse.unload(client))
                #print(f"Car {client} release resourse at time {env.now}")
                #print(f"Car {client} leave at time {env.now}")
        else:
            #print(f"Car {client} denied at time {env.now}")
            time_leave_q = env.now
            warehouse.req_den+=1
            collect_metrics(warehouse.req_den/client,"p_den",s) # collect denied requests
            
            
    else:
        warehouse.req_served_im+=1
        collect_metrics(warehouse.req_served_im/client,"p_im",s) # prob imid resolving
        
        with warehouse.cashier.request() as request:
            time_leave_q = env.now
            #print(f"Car {client} request resourse at time {env.now}")
            warehouse.cur_req_num+=1
            warehouse.req_num+=1
            collect_metrics(warehouse.cur_req_num/client,"cur_n",s)
            collect_metrics(warehouse.req_num/client,"util",s)
            yield request
            warehouse.remove_from_queue()

            yield env.process(warehouse.unload(client))
            #print(f"Car {client} release resourse at time {env.now}")
            #print(f"Car {client} leave at time {env.now}")
    
    warehouse.cur_req_num-=1
    collect_metrics(warehouse.cur_req_num/client,"cur_n",s)
    time_leave = env.now
    collect_metrics(time_leave - time_arrive ,"avrg_w",s)# time request in system
    collect_metrics(time_leave_q - time_arrive ,"avrg_w_q",s)# time in queue 
    collect_metrics(warehouse.queue_lenght,"q_length",s)# collect queue
    collect_metrics(warehouse.req_den/client,"p_den",s) # collect denied requests
    collect_metrics(warehouse.req_served_im/client,"p_im",s) # prob imid resolving
    collect_metrics(warehouse.is_free(env.now),"busy_t",s)
    
    

In [4]:
def run_warehouse(env,l,s):
    warehouse = Warehouse(env)
    car = 0
    while True:
        time = round(np.random.exponential(l),2)
        #print(f"lambda {time}")
        yield env.timeout(time)
        car +=1
        env.process(go_to_warehouse(env,car,warehouse,s))

In [5]:
import numpy as np

def collect_metrics(elem,label, metric):
    if metric.name==label:
        if not (elem==None):
            metric.collect(elem)
    
    #not Collect Относительный пропускная способность
    #not Collect Абсолютная пропускная способность 

    pass
class Statistic:
    #Avarage queue length done
    #Вероятность отказа done
    #Вероятность немедленного принятия done
    #Среднее время ожидания в очереди
    #Среднее время пребывания заявки
    #Среднее число заявок в СМО
    NAMES = ["q_length","p_den", "p_im", "avrg_w_q","avrg_w","avrg_req_num","cur_n","util","busy_t"]
    METR_NAME = {"q_length":"Avarage queue length",
                 "p_den":"Probability of refused",
                 "p_im":"Probability of imidiate responce",
                 "avrg_w_q":" Avarage time in queue",
                 "avrg_w":"Avarage time car spend in warehouse system",
                 "avrg_req_num":"Относительная пропускная способность",
                 "cur_n":"Avarage number of cars in system",
                 "util":"System utilisation",
                 "busy_t":"Mean busy period of the system"}
    def __init__(self,event,name,epsilon, num_elem):
        self.event = event
        self.name = name
        self.full_name = Statistic.METR_NAME[name]
        self.array = []
        self.num_elem = num_elem
        self.epsilon = epsilon
    def collect(self,elem):
        self.array.append(elem)
        if self.check_stability():
            self.event.succeed()
            
            
    
    def check_stability(self):
        last_index = len(self.array)-1
        if last_index-self.num_elem<0:
            return False
        first_index = last_index-self.num_elem
            
        a = np.array(self.array[first_index:last_index])
        #print(f'check_stab {a}')
        #print(f'mean {np.mean(a)}')
        #print(f"value {pow(np.var(a),0.5)}")
        if pow(float(np.var(a)),0.5) < self.epsilon:
            return True
        else:
            return False
    def calculate_avrg(self):
        a = np.array(self.array)
        return a.mean()
    def calculate(self):
        alpha = 0.9
        last_index = len(self.array)-1
        first_index = last_index-self.num_elem
        a = np.array(self.array[first_index:last_index])
        res = a[0]
        a = np.delete(a,0)
        for elem in a:
            res = alpha*elem + (1-alpha)*res
        return res


'\nЗакон распределения времени ожидания заявки\nЗакон распределения времени прибытия\n'

## Find metrics with simulation

As limit of sequence deviation of `num_elem` numbers is compared with `eps`. Simulation is repeated 1000 times.

In [8]:
#metrics = [length_of_queue, prob_den, prob_get_im, waiting_time, proc_time, av_req_num]

result = {"q_length":[[],[]],"p_den":[[],[]], "p_im":[[],[]], "avrg_w_q":[[],[]],"avrg_w":[[],[]],"avrg_req_num":[[],[]],"cur_n":[[],[]],"util":[[],[]],"busy_t":[[],[]]}
for i in range(1000):
    for [name,eps,n] in [["q_length",0.01,100],["p_den",0.01,10], ["p_im",0.1,20], ["avrg_w_q",0.01,100],["avrg_w",0.8,30],["avrg_req_num",0.3,20],["cur_n",0.001,40],["util",0.01,10],["busy_t",0.9,10]]:
        env = simpy.Environment()
        event = env.event()
        stat = Statistic(event = event,name = name,epsilon = eps, num_elem = n)
        env.process(run_warehouse(env,3,stat))
        env.run(until = event)
        result[stat.name][0].append(stat.calculate())
        result[stat.name][1].append(stat.calculate_avrg())
        #print(f"result: {stat.full_name}, {stat.calculate()}, {stat.calculate_avrg()}")

        


name, value, avarage value
result: Avarage queue length, 0.0, 0.007226665351498292
---------------------
result: Probability of refused, 0.0, 0.0
---------------------
result: Probability of imidiate responce, 1.0076457520095519, 1.0343787579795345
---------------------
result:  Avarage time in queue, 0.0, 0.0
---------------------
result: Avarage time car spend in warehouse system, 0.8070749668926949, 0.9543765730461768
---------------------
result: Относительная пропускная способность, 3.982525782847035, 3.696970245933389
---------------------
result: Avarage number of cars in system, 0.0005391987421148332, 0.008929455869172096
---------------------
result: System utilisation, 1.0, 1.0
---------------------
result: Mean busy period of the system, 2.422063420033359, 4.137387328185659
---------------------


## Print results anCompare with theoretical values

In [30]:
theor_val = {"q_length":0.870967742,"p_den":0.14516129, "p_im":0.532258065, 
             "avrg_w_q":1,"avrg_w":1,"avrg_req_num":1,"cur_n":1,"util":1,"busy_t":1}
print(f"name, value, avarage value")
for k in result.keys():
    print(f"result: {Statistic.METR_NAME[k]}, {sum(result[k][0])/len(result[k][0])}, {sum(result[k][1])/len(result[k][1])}")
    print("---------------------")
for k in result.keys():
    print(f"compare with th_solution: {Statistic.METR_NAME[k]}, {sum(result[k][0])/len(result[k][0]) - theor_val[k]} ")
    print("---------------------")

name, value, avarage value
result: Avarage queue length, 0.0, 0.007226665351498292
---------------------
result: Probability of refused, 0.0, 0.0
---------------------
result: Probability of imidiate responce, 1.0076457520095519, 1.0343787579795345
---------------------
result:  Avarage time in queue, 0.0, 0.0
---------------------
result: Avarage time car spend in warehouse system, 0.8070749668926949, 0.9543765730461768
---------------------
result: Относительная пропускная способность, 3.982525782847035, 3.696970245933389
---------------------
result: Avarage number of cars in system, 0.0005391987421148332, 0.008929455869172096
---------------------
result: System utilisation, 1.0, 1.0
---------------------
result: Mean busy period of the system, 2.422063420033359, 4.137387328185659
---------------------
compare with th_solution: Avarage queue length, -0.870967742 
---------------------
compare with th_solution: Probability of refused, -0.14516129 
---------------------
compare with 

## Calcuate value of metrics
Use convergence by probability

In [27]:
import numpy as np
for k in result.keys():
    a1 = np.array(result[k][0])
    unique = np.unique(a1)
    for i in unique:
        if (1 - a1[a1==i].sum()/len(unique))<0.01:
            print(f"{Statistic.METR_NAME[k]}, value = {i}")
            break
        

Относительная пропускная способность, value = 3.1
System utilisation, value = 1.0
